In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time

In [49]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20130101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
print(df_d.columns)
print(df_d.shape)

select * from stock_day where date>=20130101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(2304627, 11)


In [4]:
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt'] /= 1e4
df = df_d

periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [7]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

In [15]:
print(df.index.is_lexsorted())
# df.groupby('code').apply(lambda x:np.arange(1,len(x+1))).head()
df['tmp'] = 1
df['idx'] = df.reset_index('code').groupby('code')['tmp'].expanding(1).sum().astype('int')
print(df['idx'].head())
del df['tmp']

In [16]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
print(len(index))

1261


In [30]:
df_target = df.loc[index]
print(df_target.shape)
for i in range(1,6):
    df_target['idx_diff{}'.format(i)] = df_target.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    
df_target = df_target.join(df_r,how='inner',rsuffix='r_')
# 检查拼表后数据是否完整
print(df_target.shape)

(1261, 26)
idx_diff1==1: 564
idx_diff2==2: 305
idx_diff3==3: 179
idx_diff4==4: 117
idx_diff5==5: 80
(1261, 38)


In [50]:
# 统计连续超跌i天的总收益率与各年收益率情况
for i in range(1,6):
    print('idx_diff{0}=={0}'.format(i))
    print(df_target[df_target['idx_diff{}'.format(i)]==i].reset_index('code')['r'].resample('Y').agg(agg_operations))
    print('-'*5)
    print(df_target[df_target['idx_diff{}'.format(i)]==i]['r'].agg(agg_operations))
    print('-'*10+'\n')

# 统计只超跌一天的收益率情况（注意这里实际上用到了未来数据，但依然有借鉴作用）
i=1
print('idx_diff1>{}'.format(i))
print(df_target[df_target['idx_diff{}'.format(i)]>i].reset_index('code')['r'].resample('Y').agg(agg_operations))
print('-'*5)
print(df_target[df_target['idx_diff{}'.format(i)]>i]['r'].agg(agg_operations))
print('-'*10+'\n')
print(len(df_target),(df_target['idx_diff{}'.format(i)]>i).sum(),(df_target['idx_diff{}'.format(i)]==i).sum())

# 结果显示，无论是整体收益率还是19年收益率，连续超跌天数越大，则表现越差。

idx_diff1==1
                mean    median       max       min       std  size
date                                                              
2015-12-31  0.082075  0.081982  0.655405 -0.333562  0.127632   277
2016-12-31  0.058145  0.047458  0.305152 -0.066667  0.065561    58
2017-12-31 -0.007091  0.000000  0.025126 -0.038410  0.029386     5
2018-12-31  0.010027  0.000000  0.224490 -0.189268  0.061744   175
2019-12-31 -0.016409 -0.003143  0.242424 -0.554711  0.142387    39
2020-12-31 -0.046639 -0.061890  0.002283 -0.065060  0.032650    10
-----
mean        0.048365
median      0.038402
max         0.655405
min        -0.554711
std         0.111786
size      564.000000
Name: r, dtype: float64
----------

idx_diff2==2
                mean    median       max       min       std  size
date                                                              
2015-12-31  0.058576  0.056522  0.551485 -0.221648  0.103748   107
2016-12-31  0.066191  0.055258  0.203170 -0.049231  0.061458    28
20

In [44]:
print(df_target[df_target['idx_diff{}'.format(5)]==5][['open','close','amt','r','idx_diff5']])

                           open      close         amt         r  idx_diff5
code      date                                                             
000155.SZ 2018-01-02    9.45504    9.61184   24.836336 -0.006494        5.0
000750.SZ 2015-08-31   71.57995   70.48990   66.097533  0.031579        5.0
002434.SZ 2018-10-24   38.40280   38.46800    4.111978  0.080000        5.0
          2018-10-25   37.49000   37.81600    4.406924  0.054329        5.0
          2018-10-26   38.40280   37.62040    2.918031  0.068847        5.0
          2018-10-29   37.88120   37.03360    2.669241  0.085664        5.0
          2018-10-30   37.29440   37.81600    3.678458  0.095238        5.0
002470.SZ 2019-05-14   18.27132   18.61284   18.988017 -0.036613        5.0
          2019-05-15   18.65553   19.21050   25.548514 -0.058166        5.0
          2019-05-16   19.08243   19.16781   17.851994 -0.088692        5.0
          2019-05-17   19.25319   18.35670   18.346684  0.019002        5.0
          20